In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup_backend import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend()

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'reset', 'x', 'sx', 'measure', 'cz']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4

# Load parameters
with open(f"results/get_param/{n_qubits}Q_param.json", "r") as f:
    param = json.load(f)
C = param["C"]
K = param["K"]
n_features = (
    K + 1
)  # 実数部分の特徴量の数。1 は t=0, つまり定数項。虚数も含んだ特徴量の数は 2K+1

print(f"C: {C}")
print(f"K: {K}")

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q_optimal_n_step.json", orient="records"
)
times = df_n_step["t"].values
print(f"times: {times}")
display(df_n_step)

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q_dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

C: 9
K: 8
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268 ]


,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.974201
2,0.698132,2,0.950947
3,1.047198,2,0.946502
4,1.396263,3,0.923480
5,1.745329,3,0.911794
6,2.094395,4,0.892342
7,2.443461,5,0.876343
8,2.792527,5,0.860245


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899247,13.899220,2.726160e-05
1,1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.586491,28.585213,1.278298e-03
2,2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653078,2.653082,3.712600e-06
3,3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222994,7.222891,1.023508e-04
4,4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865096,0.865078,1.800550e-05
5,5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138947,1.138950,2.786300e-06
6,6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060702,9.060659,4.298700e-05
7,7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.366950,4.367027,7.663200e-05
8,8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566096,4.566077,1.851980e-05
9,9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720388,3.720380,8.204800e-06


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [7]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noisy,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [8]:
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [10]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,0.964844,0.682974,0.320415,0.163168,0.347402,-0.096882,0.049051,-0.048373,-0.020284,-0.192801,0.133068,-0.491043,0.191827,-0.623817,-0.053486,-0.400065,-0.296825,13.899247
1,1,0.959961,0.513544,0.488436,-0.140906,0.291099,-0.230253,-0.145012,0.123488,-0.240471,0.347472,-0.109459,0.310902,0.030349,0.144420,-0.066022,0.088640,-0.018483,28.586491
2,2,0.968750,0.737375,-0.220582,0.275270,-0.260011,-0.048591,-0.259628,-0.046219,-0.215493,0.157905,-0.273088,0.291995,-0.342558,0.152151,-0.341102,-0.169612,-0.132792,2.653078
3,3,0.952148,0.711584,0.265191,0.248141,0.256195,-0.188601,-0.032796,-0.351816,-0.444075,-0.188097,-0.755244,0.124959,-0.608663,0.289821,-0.238562,0.184566,0.128136,7.222994
4,4,0.967773,0.910069,-0.131466,0.803615,-0.256547,0.654933,-0.377733,0.523307,-0.468253,0.346815,-0.503547,0.222413,-0.525484,0.167143,-0.478572,0.085440,-0.381198,0.865096
5,5,0.961914,0.614361,-0.340044,0.088465,-0.216266,-0.062665,0.118351,-0.022113,0.068910,-0.245251,-0.126996,-0.468618,-0.014697,-0.202326,0.153915,0.137378,-0.036936,1.138947
6,6,0.959961,0.752447,0.231609,0.374160,0.232415,0.138405,0.051560,0.139481,-0.003084,0.150103,0.178002,-0.190594,0.418369,-0.454601,0.372915,-0.501802,0.170278,9.060702
7,7,0.960938,0.787609,-0.125540,0.437928,-0.274337,0.111841,-0.313592,0.118425,-0.261834,0.246698,0.000805,0.352275,0.246984,0.326663,0.246444,0.273009,0.145838,4.366950
8,8,0.970703,0.801576,0.323058,0.461630,0.476024,0.074246,0.462240,-0.214279,0.240954,-0.260736,-0.069646,-0.121749,-0.286554,0.197641,-0.432954,0.588114,-0.338450,4.566096
9,9,0.973633,0.718293,0.039776,0.303435,0.183935,-0.026839,0.261211,0.077781,0.257561,0.245845,0.061020,0.367778,-0.209116,0.264732,-0.248505,0.159674,-0.086660,3.720388


In [9]:
# Create circuits
# 1 batch - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Prepare circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        # n_steps = get_n_steps(times[k])
        n_step = df_n_step[df_n_step["t"] == times[k]]["n_step"].values[0]

        circuit_phase0 = heisenberg.get_circuit(times[k], n_step, phase=0)
        circuit_phase1 = heisenberg.get_circuit(times[k], n_step, phase=1)
        circuit_phase2 = heisenberg.get_circuit(times[k], n_step, phase=2)
        circuit_phase3 = heisenberg.get_circuit(times[k], n_step, phase=3)

        exec_circuit_phase0 = transpile(circuit_phase0, backend_sim_noiseless)
        exec_circuit_phase1 = transpile(circuit_phase1, backend_sim_noiseless)
        exec_circuit_phase2 = transpile(circuit_phase2, backend_sim_noiseless)
        exec_circuit_phase3 = transpile(circuit_phase3, backend_sim_noiseless)

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Calculating features for sample 0/55


Calculating features for sample 1/55
Calculating features for sample 2/55
Calculating features for sample 3/55
Calculating features for sample 4/55
Calculating features for sample 5/55
Calculating features for sample 6/55
Calculating features for sample 7/55
Calculating features for sample 8/55
Calculating features for sample 9/55
Calculating features for sample 10/55
Calculating features for sample 11/55
Calculating features for sample 12/55
Calculating features for sample 13/55
Calculating features for sample 14/55
Calculating features for sample 15/55
Calculating features for sample 16/55
Calculating features for sample 17/55
Calculating features for sample 18/55
Calculating features for sample 19/55
Calculating features for sample 20/55
Calculating features for sample 21/55
Calculating features for sample 22/55
Calculating features for sample 23/55
Calculating features for sample 24/55
Calculating features for sample 25/55
Calculating features for sample 26/55
Calculating features 

In [10]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim_noiseless):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/55
Running circuits for sample 1/55
Running circuits for sample 2/55
Running circuits for sample 3/55
Running circuits for sample 4/55
Running circuits for sample 5/55
Running circuits for sample 6/55
Running circuits for sample 7/55
Running circuits for sample 8/55
Running circuits for sample 9/55
Running circuits for sample 10/55
Running circuits for sample 11/55
Running circuits for sample 12/55
Running circuits for sample 13/55
Running circuits for sample 14/55
Running circuits for sample 15/55
Running circuits for sample 16/55
Running circuits for sample 17/55
Running circuits for sample 18/55
Running circuits for sample 19/55
Running circuits for sample 20/55
Running circuits for sample 21/55
Running circuits for sample 22/55
Running circuits for sample 23/55
Running circuits for sample 24/55
Running circuits for sample 25/55
Running circuits for sample 26/55
Running circuits for sample 27/55
Running circuits for sample 28/55
Running circuits for sam

In [12]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_sim = pd.DataFrame(data_sim, columns=columns)
display(df_sim)

# Save the simulation data
path = "results/fourier_feature_sim"
df_sim.to_json(f"{path}/feature_sim_noiseless.json", orient="records", indent=4)
with open(f"{path}/probs_phase0_sim_noiseless.json", "w") as f:
    json.dump(probs_phase0_sim, f, indent=4)

with open(f"{path}/probs_phase1_sim_noiseless.json", "w") as f:
    json.dump(probs_phase1_sim, f, indent=4)

with open(f"{path}/probs_phase2_sim_noiseless.json", "w") as f:
    json.dump(probs_phase2_sim, f, indent=4)

with open(f"{path}/probs_phase3_sim_noiseless.json", "w") as f:
    json.dump(probs_phase3_sim, f, indent=4)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,1.0,0.706114,0.375307,0.213255,0.321226,-0.078159,0.028074,-0.078442,-0.011719,-0.277071,0.152880,-0.618868,0.229090,-0.813039,-0.070392,-0.506908,-0.380469,13.899220
1,1,1.0,0.548074,0.518690,-0.177835,0.306601,-0.298964,-0.120413,0.164723,-0.322467,0.386129,-0.112398,0.361931,-0.012191,0.199151,-0.066338,0.066297,-0.045714,28.585213
2,2,1.0,0.798557,-0.227124,0.310811,-0.327270,-0.063735,-0.252757,-0.071320,-0.223143,0.135842,-0.323222,0.324523,-0.383766,0.194756,-0.364917,-0.192686,-0.193005,2.653082
3,3,1.0,0.780524,0.313610,0.267600,0.281358,-0.222763,-0.057137,-0.355453,-0.546363,-0.170823,-0.816208,0.169378,-0.726435,0.371905,-0.293285,0.236820,0.125050,7.222891
4,4,1.0,0.963521,-0.157379,0.891336,-0.306115,0.721329,-0.435237,0.527981,-0.575506,0.388149,-0.601289,0.310785,-0.642202,0.175477,-0.565267,0.168088,-0.448392,0.865078
5,5,1.0,0.649238,-0.413218,0.077597,-0.196780,-0.094918,0.092015,-0.024808,0.056088,-0.303801,-0.132463,-0.549042,-0.009270,-0.329695,0.175114,0.161920,-0.064582,1.138950
6,6,1.0,0.796614,0.267551,0.402553,0.229713,0.154019,0.037945,0.201685,-0.008138,0.122291,0.197506,-0.157928,0.510592,-0.545015,0.456502,-0.607982,0.228628,9.060659
7,7,1.0,0.822635,-0.146970,0.468147,-0.307185,0.185040,-0.354731,0.174050,-0.297389,0.307787,0.016637,0.407578,0.297709,0.385245,0.308603,0.328509,0.170909,4.367027
8,8,1.0,0.869803,0.304427,0.514823,0.565772,0.094597,0.526839,-0.238243,0.306222,-0.346433,-0.019040,-0.114129,-0.323921,0.264039,-0.490373,0.655381,-0.480453,4.566077
9,9,1.0,0.779745,0.034774,0.297242,0.209769,-0.015450,0.301629,0.089065,0.304836,0.276429,0.076773,0.416139,-0.289103,0.355037,-0.291594,0.196299,-0.104519,3.720380


## Trotter simulation (Simulator, Noisy)

In [13]:
# Create circuits
# 1 batch - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        # n_steps = get_n_steps(times[k])
        n_step = df_n_step[df_n_step["t"] == times[k]]["n_step"].values[0]

        circuit_phase0 = heisenberg.get_circuit(times[k], n_step, phase=0)
        circuit_phase1 = heisenberg.get_circuit(times[k], n_step, phase=1)
        circuit_phase2 = heisenberg.get_circuit(times[k], n_step, phase=2)
        circuit_phase3 = heisenberg.get_circuit(times[k], n_step, phase=3)

        exec_circuit_phase0 = transpile(circuit_phase0, backend_sim_noisy)
        exec_circuit_phase1 = transpile(circuit_phase1, backend_sim_noisy)
        exec_circuit_phase2 = transpile(circuit_phase2, backend_sim_noisy)
        exec_circuit_phase3 = transpile(circuit_phase3, backend_sim_noisy)

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Calculating features for sample 0/55
Calculating features for sample 1/55
Calculating features for sample 2/55
Calculating features for sample 3/55
Calculating features for sample 4/55
Calculating features for sample 5/55
Calculating features for sample 6/55
Calculating features for sample 7/55
Calculating features for sample 8/55
Calculating features for sample 9/55
Calculating features for sample 10/55
Calculating features for sample 11/55
Calculating features for sample 12/55
Calculating features for sample 13/55
Calculating features for sample 14/55
Calculating features for sample 15/55
Calculating features for sample 16/55
Calculating features for sample 17/55
Calculating features for sample 18/55
Calculating features for sample 19/55
Calculating features for sample 20/55
Calculating features for sample 21/55
Calculating features for sample 22/55
Calculating features for sample 23/55
Calculating features for sample 24/55
Calculating features for sample 25/55
Calculating features f

In [14]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim_noiseless):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/55
Running circuits for sample 1/55
Running circuits for sample 2/55
Running circuits for sample 3/55
Running circuits for sample 4/55
Running circuits for sample 5/55
Running circuits for sample 6/55
Running circuits for sample 7/55
Running circuits for sample 8/55
Running circuits for sample 9/55
Running circuits for sample 10/55
Running circuits for sample 11/55
Running circuits for sample 12/55
Running circuits for sample 13/55
Running circuits for sample 14/55
Running circuits for sample 15/55
Running circuits for sample 16/55
Running circuits for sample 17/55
Running circuits for sample 18/55
Running circuits for sample 19/55
Running circuits for sample 20/55
Running circuits for sample 21/55
Running circuits for sample 22/55
Running circuits for sample 23/55
Running circuits for sample 24/55
Running circuits for sample 25/55
Running circuits for sample 26/55
Running circuits for sample 27/55
Running circuits for sample 28/55
Running circuits for sam

In [16]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_sim = pd.DataFrame(data_sim, columns=columns)
display(df_sim)

# Save the simulation data
path = "results/fourier_feature_sim"
df_sim.to_json(f"{path}/feature_sim_noisy.json", orient="records", indent=4)
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim, f, indent=4)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim, f, indent=4)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim, f, indent=4)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim, f, indent=4)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,1.0,0.762443,0.390774,0.219918,0.301806,-0.066307,0.037971,-0.100064,-0.056115,-0.208400,0.162605,-0.597200,0.217222,-0.818461,-0.075483,-0.491596,-0.367544,13.899220
1,1,1.0,0.554864,0.528341,-0.154230,0.302606,-0.314515,-0.145242,0.116837,-0.282196,0.408194,-0.076577,0.361957,-0.010007,0.191075,-0.088769,0.073500,-0.012819,28.585213
2,2,1.0,0.805549,-0.210274,0.283580,-0.363750,-0.056023,-0.273511,-0.061820,-0.277999,0.152797,-0.327964,0.315845,-0.362774,0.175323,-0.355912,-0.234724,-0.177719,2.653082
3,3,1.0,0.767661,0.291075,0.225163,0.301857,-0.210771,-0.053869,-0.361595,-0.540001,-0.176058,-0.829643,0.214819,-0.699677,0.373731,-0.284254,0.232181,0.154982,7.222891
4,4,1.0,0.969251,-0.162120,0.864120,-0.297868,0.735042,-0.418243,0.558861,-0.567936,0.422811,-0.602339,0.270026,-0.617420,0.201116,-0.574968,0.139691,-0.468957,0.865078
5,5,1.0,0.654839,-0.360410,0.120942,-0.198152,-0.031365,0.081105,-0.054989,0.044133,-0.291418,-0.152742,-0.569872,-0.025901,-0.320176,0.197168,0.186186,-0.077812,1.138950
6,6,1.0,0.786622,0.257434,0.435218,0.194963,0.158034,0.007876,0.195341,-0.014890,0.181701,0.235429,-0.171679,0.495887,-0.558174,0.496007,-0.622511,0.185453,9.060659
7,7,1.0,0.809955,-0.135564,0.465281,-0.288164,0.166077,-0.334459,0.156480,-0.263373,0.292353,0.016196,0.377182,0.311389,0.396964,0.314542,0.352797,0.187551,4.367027
8,8,1.0,0.846253,0.329826,0.482781,0.573850,0.054153,0.519739,-0.251339,0.284186,-0.297941,-0.021668,-0.137556,-0.326938,0.262423,-0.481679,0.632759,-0.433534,4.566077
9,9,1.0,0.796249,0.031631,0.349795,0.202137,-0.016758,0.329392,0.058139,0.250489,0.269407,0.062679,0.413283,-0.271324,0.307626,-0.301741,0.244737,-0.111343,3.720380
